In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
# read dataset into datasets
from datasets import Dataset
import pandas as pd
dataset_name = "five-star-trajectories"
dataset_path = Path(f"lm_act_eval/.cache/{dataset_name}")
traj_df = pd.read_csv(dataset_path/"csv/data.csv")
traj_dataset = Dataset.from_pandas(traj_df)
print(traj_df.columns)
# traj_df[traj_df.session_id==traj_df.session_id.unique()[0]]
# traj_df.head(2)

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['Unnamed: 0', '_id', 'tenant', 'project_id', 'created_at', 'updated_at',
       'inputs', 'history', 'ground_truth', 'linked_event', 'linked_evals',
       'linked_datasets', 'saved', 'type', 'metadata', 'QUERY', 'URL', 'DOM',
       'RULES', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'USER_CONTEXT', 'PREV_ACTIONS', 'prompt', 'CHAT_HISTORY'],
      dtype='object')


In [13]:
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

In [6]:
# for i in [
#   traj_df.PREV_ACTIONS,
#   traj_df.previousActionsRepetitionPrompt,
#   traj_df.chat_completion_messages,
#   traj_df.ground_truth,
#   traj_df.screenshot
#   ]:
#   if type(i[2])==str:
#     print(i[2].replace('\n\n','\n'))
#   else:
#     print(i[2])
#   print("")
  

In [51]:
from typing import Callable
from tqdm import tqdm

In [56]:
# subdependent 
from lm_act_eval.evaluation_harness.evaluators.metrics.base import DataFrameEvaluator
from lm_act_eval.evaluation_harness.helper_functions import multion

In [57]:
from lm_act_eval.evaluation_harness.openai.vision.gptv import GPTV
from lm_act_eval.evaluation_harness.openai.vision.config import gptv_config

from lm_act_eval.evaluation_harness.openai.vision.evaluator import GPTVEvaluator

In [58]:
from lm_act_eval._init_conf import init_conf
hydra_cfg = init_conf('trajectory_eval')
hydra_cfg

{'sft': {'trajectory': {'data': {'path': 'lm_act_eval/.cache/five-star-trajectories/csv/data+gptv-eligible.csv', 'columns': {'y': 'ground_truth', "y_'": 'GPTV_generations'}, 'extract_fs': {'USER_QUERY': {'QUERY': 'parse_completion.parse_query'}, 'GOAL': {'chat_completion_messages': 'parse_completion.parse_content'}, 'action': {'action': 'extract_action'}, 'thought': {'thought': 'extract_thought'}, 'explanation': {'explanation': 'extract_explanation'}}, 'logging': {'wandb': {'project': 'opentable', 'result': 'lm_act_eval-run'}, 'hugging': {'space': 'multion-agi'}}}, 'comparator': {'gptv': {'model': 'gpt-4-vision-preview', 'max_token': 300, 'img_fidelity': 'high'}}, 'metrics': {'gptv': ['GOAL', 'QUERY'], 'edit_distance': ['actions'], 'last_state': ['URL'], 'bleu': ['explanation'], 'llm_relevancy': ['explanation']}}}}

In [63]:
# original 
df = pd.read_csv(hydra_cfg.sft.trajectory.data.path)
df.chat_completion_messages.iloc[0][:100]

"[{'role': 'system', 'content': '**Goal**: Let\\'s play a game - You are an expert agent named MULTI·O"

In [61]:
gptv_evaluator = GPTVEvaluator(
  hydra_cfg.sft.trajectory.data)

USER_QUERY QUERY <bound method ParseChatCompletion.parse_query of <lm_act_eval.evaluation_harness.helper_functions.multion.ParseChatCompletion object at 0x7efef137f690>>
GOAL chat_completion_messages <bound method ParseChatCompletion.parse_content of <lm_act_eval.evaluation_harness.helper_functions.multion.ParseChatCompletion object at 0x7efef1101490>>


/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:152: UserWarning: Unexpected format, returning suspicious content of type <class 'str'>
  


ValueError: not enough values to unpack (expected 2, got 1)

In [65]:
# Using as agent
from lm_act_eval.evaluation_harness.helper_functions.multion import extract_explanation

df.chat_completion_messages.apply( ParseChatCompletion().parse_content)

/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:152: UserWarning: Unexpected format, returning suspicious content of type <class 'str'>
  


pandas.core.series.Series

In [19]:
# evaluator prompt
from lm_act_eval.evaluation_harness.openai.vision.prompts import DEFAULT_EVAL_PROMPT
eval_query = eval_input_df.apply(lambda r: DEFAULT_EVAL_PROMPT.format(**r), axis=1)


In [20]:
input_df = pd.DataFrame(index=eligible_traj_df.index)
input_df['text'] = eval_query
input_df['images'] = screenshots

In [3]:
import re
evals = input_df.iloc[:5].progress_apply(lambda r: gptv_pipeline.generate_completion(**r), axis=1)

NameError: name 'input_df' is not defined

In [62]:
evals

Unnamed: 0
169    SCORE: 90\nEXPLANATION:\nThe user is very clos...
170    SCORE: 95\nEXPLANATION:\nThe user is very clos...
171    SCORE: 90\nEXPLANATION:\nThe user has successf...
172    SCORE: 100\nEXPLANATION:\nThe user has success...
408    SCORE: 95\nEXPLANATION:\nThe user is very clos...
dtype: object

In [ ]:
result_df = evals.str.split('\n', n=1, expand=True).rename_axis(index=None)
result_df.columns = ['Score', 'Explanation']
result_df.head(2)

In [2]:
result_df['Score'] = result_df['Score'].apply(lambda s: extract_first(s, term='SCORE'))
extract_explanation = lambda x: re.search(r'EXPLANATION:\n(.+)', x, re.DOTALL).group(1) if re.search(r'EXPLANATION:\n', x) else None
result_df['Explanation'] = result_df['Explanation'].apply(lambda s: extract_explanation(s))

NameError: name 'result_df' is not defined

In [111]:
print(result_df.Explanation.iloc[0])

The user is very close to completing the task. They have successfully navigated to the Google search query page that might contain links to the NIH website where the information about the clinical trial can be found. However, the user has not yet clicked on the relevant links that would take them directly to the NIH website to check the actual enrollment count of the clinical trial on H. pylori in acne vulgaris patients for the specified period.

The next step would be to follow the links provided in the search results to the clinicaltrials.gov entries that appear to contain the desired information. Since the exact URL or the identity of the clickable link in the search results that leads directly to the information on the NIH website is not visible in the provided context, it can't be considered as completely done. A little more navigation within the search results is needed to reach the exact page that holds the enrollment count.

STATUS: CONTINUE

COMMANDS:
CLICK 1
EXPLANATION: I am

In [ ]:
eligible_traj_df.loc[:, 'GPTV_evaluations'] = completion_responses

## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,GPTV response,chat_completion_messages_content,GPTV_evaluation
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,It appears that you are looking for informatio...,**Goal**: Let's play a game - You are an exper...,SCORE: 60\nEXPLANATION:\nThe user appears to b...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,The image shows a webpage from ClinicalTrials....,**Goal**: Let's play a game - You are an exper...,SCORE: 95\nEXPLANATION:\nThe user is very clos...


Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories-gptv_eval>